In [1]:
# Import necessary libraries
import pandas as pd
import requests
import datetime
import csv
import pytz

In [2]:
# Check if request was successful
r = requests.get('https://waitz.io/live/ucsd')
r.status_code

200

In [3]:
# Read in data
d = r.json()['data']
d

[{'name': '64 Degrees',
  'id': 54,
  'busyness': 0,
  'people': 0,
  'isAvailable': True,
  'capacity': 175,
  'hourSummary': 'open',
  'isOpen': True,
  'bestLabel': None,
  'bestLocations': [],
  'percentage': 0,
  'locHtml': {'initial': '\n    <div class="row spaced loc-section not-busy" id="loc-54">\n        <h2 class="title">\n            64 Degrees\n        </h2>\n        <p class="loc-summary">\n            Not Busy (0%)\n        </p>\n    </div>    \n    ',
   'summary': 'Not Busy (0%)',
   'class': 'not-busy'},
  'subLocs': False},
 {'name': '6th Restaurant',
  'id': 70,
  'busyness': 4,
  'people': 14,
  'isAvailable': True,
  'capacity': 350,
  'hourSummary': 'open',
  'isOpen': True,
  'bestLabel': None,
  'bestLocations': [],
  'percentage': 0.04,
  'locHtml': {'initial': '\n    <div class="row spaced loc-section not-busy" id="loc-70">\n        <h2 class="title">\n            6th Restaurant\n        </h2>\n        <p class="loc-summary">\n            Not Busy (4%)\n      

In [4]:
# Create dataframe from JSON & get relevant columns for data analysis
df = pd.DataFrame(d)[['name', 'id', 'busyness', 'people', 'isAvailable', 'capacity', 'hourSummary', 'isOpen', 'bestLocations', 'percentage', 'subLocs']]

In [5]:
# Create timestamp
pst = pytz.timezone('America/Los_Angeles')

timestamp = datetime.datetime.now(tz=pst).strftime("%Y-%m-%d %H:%M")

In [6]:
# Gets Geisel, both gyms, and Biomedical Library (Filtering data essentially & change ids)

df = df[(df['name'] == 'Geisel Library') | 
        (df['name'] == 'Main Gym') | 
        (df['name'] == 'Price Center') | 
        (df['name'] == 'WongAvery Library') | 
        (df['name'] == 'RIMAC Fitness Gym') |
        (df['name'] == 'WongAvery Grad Study')].reset_index().drop(columns=['index'])
# Change ids such that they are now unique
df.at[0, 'id'] = 16
df.at[2, 'id'] = 18
df.at[5, 'id'] = 19
df

,name,id,busyness,people,isAvailable,capacity,hourSummary,isOpen,bestLocations,percentage,subLocs
0,Geisel Library,16,12,326,True,2785,Partially Open,True,"[{'abbreviation': '2E', 'id': 3, 'busyness': 1...",0.12,"[{'busyness': 4, 'people': 6, 'capacity': 150,..."
1,Main Gym,13,21,11,True,50,open,True,[],0.21,False
2,Price Center,18,25,161,True,650,open,True,"[{'abbreviation': 'W', 'id': 10, 'busyness': 6}]",0.25,"[{'busyness': 33, 'people': 150, 'capacity': 4..."
3,RIMAC Fitness Gym,12,47,58,True,125,open,True,[],0.47,False
4,WongAvery Grad Study,17,3,1,True,35,open,True,[],0.03,False
5,WongAvery Library,19,1,8,True,748,open,True,"[{'abbreviation': '2', 'id': 15, 'busyness': 1}]",0.01,"[{'busyness': 2, 'people': 4, 'capacity': 190,..."


In [7]:
# Add timestamp column
df['timestamp'] = timestamp

In [8]:
with_sublocs = df[df['subLocs'] != False]
with_sublocs

,name,id,busyness,people,isAvailable,capacity,hourSummary,isOpen,bestLocations,percentage,subLocs,timestamp
0,Geisel Library,16,12,326,True,2785,Partially Open,True,"[{'abbreviation': '2E', 'id': 3, 'busyness': 1...",0.12,"[{'busyness': 4, 'people': 6, 'capacity': 150,...",2023-07-18 20:51
2,Price Center,18,25,161,True,650,open,True,"[{'abbreviation': 'W', 'id': 10, 'busyness': 6}]",0.25,"[{'busyness': 33, 'people': 150, 'capacity': 4...",2023-07-18 20:51
5,WongAvery Library,19,1,8,True,748,open,True,"[{'abbreviation': '2', 'id': 15, 'busyness': 1}]",0.01,"[{'busyness': 2, 'people': 4, 'capacity': 190,...",2023-07-18 20:51


In [10]:
other_locs = df[df['subLocs'] == False]
other_locs

,name,id,busyness,people,isAvailable,capacity,hourSummary,isOpen,bestLocations,percentage,subLocs,timestamp
1,Main Gym,13,46,23,True,50,open,True,[],0.46,False,2023-07-11 17:13
3,RIMAC Fitness Gym,12,71,89,True,125,open,True,[],0.71,False,2023-07-11 17:13
4,WongAvery Grad Study,17,15,5,True,35,open,True,[],0.15,False,2023-07-11 17:13


In [11]:
geisel_cap = pd.DataFrame(with_sublocs.loc[0].subLocs).loc[:8][['id', 'name', 'capacity']]

pd.DataFrame(with_sublocs.iloc[0].subLocs).loc[:8]

,busyness,people,capacity,isAvailable,id,name,abbreviation,hourSummary,isOpen,percentage,liveHTML,subLocHtml,updateHtml
0,22,33,150,True,1,1st Floor East,1E,open,True,0.22,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
1,11,55,520,True,2,1st Floor West,1W,open,True,0.11,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
2,21,127,620,True,3,2nd Floor East,2E,open,True,0.21,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
3,37,171,460,True,4,2nd Floor West,2W,open,True,0.37,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
4,9,7,80,True,5,4th Floor,4,open,True,0.09,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
5,24,37,155,True,6,5th Floor,5,open,True,0.24,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
6,27,120,440,True,7,6th Floor,6,open,True,0.27,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
7,42,82,195,True,8,7th Floor,7,open,True,0.42,"\n <div class=""zone-card not-busy"">\n ...","{'initial': ' <div class=""row spaced not-b...","{'busyness': 33, 'max': 'hi'}"
8,85,141,165,True,9,8th Floor,8,open,True,0.85,"\n <div class=""zone-card very-busy"">\n ...","{'initial': ' <div class=""row spaced very-...","{'busyness': 33, 'max': 'hi'}"


In [11]:
price_cap = pd.DataFrame(with_sublocs.loc[2].subLocs).loc[:1].sort_values(by='id')[['id', 'name', 'capacity']]

In [12]:
biomed_cap = pd.DataFrame(with_sublocs.loc[5].subLocs)[['id', 'name', 'capacity']]

In [14]:
# Change ids such that they are now distinct
capacities = df[['id', 'name', 'capacity']]
capacities.at[0, 'id'] = 16
capacities.at[2, 'id'] = 18
capacities.at[5, 'id'] = 19
capacities = capacities.sort_values(by='id')

In [54]:
cap_table = pd.concat([geisel_cap, price_cap, biomed_cap, capacities]).sort_values(by='id').set_index('id')
cap_table.to_csv('capacity_table.csv')

# Creating CSVs

In [12]:
import os

from io import StringIO

In [9]:
fieldnames = ['timestamp', 'busyness', 'people', 'id', 'percentage']

In [20]:
# Create script for just Geisel
geisel_df = df[df['name'] == 'Geisel Library']

geisel_fp = 'geisel/geisel_main.csv'
if os.path.exists(geisel_fp):
    with open(geisel_fp, 'a') as geisel:
        fieldnames = fieldnames

        csv_writer = csv.DictWriter(geisel, fieldnames=fieldnames)

        csv_writer.writerow(geisel_df.iloc[0][fieldnames].to_dict())
else:
    with open(geisel_fp, 'w') as geisel:
        csv_writer = csv.DictWriter(geisel, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerow(geisel_df.iloc[0][fieldnames].to_dict())

geisel_floors = pd.DataFrame(geisel_df['subLocs'].iloc[0])
geisel_floors['timestamp'] = timestamp

for i, row in geisel_floors.iterrows():
    floor = row['abbreviation']
    floor_fp = f'geisel/geisel_{floor}.csv'
    if os.path.exists(floor_fp):
        with open(floor_fp, 'a') as geisel:
            csv_writer = csv.DictWriter(geisel, fieldnames=fieldnames)

            csv_writer.writerow(row[fieldnames].to_dict())
    else:
        with open(floor_fp, 'w') as geisel:

            csv_writer = csv.DictWriter(geisel, fieldnames=fieldnames)

            csv_writer.writeheader()
            csv_writer.writerow(row[fieldnames].to_dict())

In [21]:
# Create script for Biomed
biomed_df = df[df['name'] == 'WongAvery Library']

biomed_fp = 'biomed/biomed_main.csv'
if os.path.exists(biomed_fp):
    with open(biomed_fp, 'a') as biomed:
        csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

        csv_writer.writerow(biomed_df.iloc[0][fieldnames].to_dict())
else:
    with open(biomed_fp, 'w') as biomed:
        csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerow(biomed_df.iloc[0][fieldnames].to_dict())

biomed_floors = pd.DataFrame(biomed_df['subLocs'].iloc[0])
biomed_floors['timestamp'] = timestamp

for i, row in biomed_floors.iterrows():
    floor = row['abbreviation']
    floor_fp = f'biomed/biomed_{floor}.csv'
    if os.path.exists(floor_fp):
        with open(floor_fp, 'a') as biomed:
            csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

            csv_writer.writerow(row[fieldnames].to_dict())
    else:
        with open(floor_fp, 'w') as biomed:

            csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

            csv_writer.writeheader()
            csv_writer.writerow(row[fieldnames].to_dict())

In [22]:
# Create script for Price
price_df = df[df['name'] == 'Price Center']

price_fp = 'price/price_main.csv'
if os.path.exists(price_fp):
    with open(price_fp, 'a') as biomed:
        csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

        csv_writer.writerow(geisel_df.iloc[0][fieldnames].to_dict())
else:
    with open(price_fp, 'w') as biomed:
        csv_writer = csv.DictWriter(biomed, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerow(biomed_df.iloc[0][fieldnames].to_dict())

price_floors = pd.DataFrame(price_df['subLocs'].iloc[0])
price_floors['timestamp'] = timestamp

for i, row in price_floors.iterrows():
    floor = row['abbreviation']
    floor_fp = f'price/price_{floor}.csv'
    if os.path.exists(floor_fp):
        with open(floor_fp, 'a') as price:
            csv_writer = csv.DictWriter(price, fieldnames=fieldnames)

            csv_writer.writerow(row[fieldnames].to_dict())
    else:
        with open(floor_fp, 'w') as price:

            csv_writer = csv.DictWriter(price, fieldnames=fieldnames)

            csv_writer.writeheader()
            csv_writer.writerow(row[fieldnames].to_dict())

In [23]:
# Code for RIMAC

rimac_df = df[df['name'] == 'RIMAC Fitness Gym'][fieldnames]
rimac_fp = 'gyms/rimac.csv'
# Check if it exists to decide whether or not to write header
if os.path.exists(rimac_fp):
    with open(rimac_fp, 'a') as main:
        fieldnames = fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=fieldnames)

        csv_writer.writerow(rimac_df.iloc[0].to_dict())
else:
    with open(rimac_fp, 'w') as main:
        csv_writer = csv.DictWriter(main, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerow(rimac_df.iloc[0].to_dict())


In [24]:
# Code for Main Gym

mg_df = df[df['name'] == 'Main Gym'][fieldnames]
mg_fp = 'gyms/main_gym.csv'

# Check if it exists to decide whether or not to write header
if os.path.exists(mg_fp):
    with open(mg_fp, 'a') as main:
        fieldnames = fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=fieldnames)

        csv_writer.writerow(mg_df.iloc[0].to_dict())
else:
    with open(mg_fp, 'w') as main:
        csv_writer = csv.DictWriter(main, fieldnames=fieldnames)

        csv_writer.writeheader()
        csv_writer.writerow(mg_df.iloc[0].to_dict())


### Best Locations for Each Location with Sublocations

In [25]:
bl_fieldnames = ['abbreviation']

In [37]:
# Geisel

best_geisel_fp = 'bestLocs/geisel.csv'
if os.path.exists(best_geisel_fp):
    with open(best_geisel_fp, 'a') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writerows([{'abbreviation': i['abbreviation']} for i in geisel_df['bestLocations'].iloc[0]])
        print('complete')
else:
    with open(best_geisel_fp, 'w') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writeheader()

        csv_writer.writerows([{'abbreviation': i['abbreviation']} for i in geisel_df['bestLocations'].iloc[0]])
        print('write')

complete


In [38]:
# Price
best_price_fp = 'bestLocs/price.csv'
if os.path.exists(best_price_fp):
    with open(best_price_fp, 'a') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writerow({'abbreviation': price_df['bestLocations'].iloc[0][0]['abbreviation']})
else:
    with open(best_price_fp, 'w') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writeheader()

        csv_writer.writerow({'abbreviation': price_df['bestLocations'].iloc[0][0]['abbreviation']})

In [34]:
# Biomed/WongAvery
best_biomed_fp = 'bestLocs/biomed.csv'
if os.path.exists(best_biomed_fp):
    with open(best_biomed_fp, 'a') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writerow({'abbreviation': biomed_df['bestLocations'].iloc[0][0]['abbreviation']})
else:
    with open(best_biomed_fp, 'w') as main:
        fieldnames = bl_fieldnames

        csv_writer = csv.DictWriter(main, fieldnames=bl_fieldnames)

        csv_writer.writeheader()

        csv_writer.writerow({'abbreviation': biomed_df['bestLocations'].iloc[0][0]['abbreviation']})